<a href="https://colab.research.google.com/github/hectorpilo/bootcamp-ds-sonda/blob/main/CORE_Analisis_Ventas_Autos_Usados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prediccion de Predio de Venta de Autos Usados**
* El presente notebook, muestra el proceso de analisis de un dataset de autos usados. Información de vehiculos que fue recolectada por varios usarios y dispuestas en Kaggle para su analisis.
* Eliminé varias columnas que no me servirian para el entrenamiento, como el numero de vin, urls, entre otras.
* Realicé analisis visuales univariados y multivariados.
* hice entrenamientos con outliers y sin outliers.
* Realicé una funcion que solicita al usuario un input de año, kilometraje, marca, tipo de combustible, condicion de uso y tipo de carroceria para poder predecir ambos modelos. Con Outliers y sin Outliers.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/BBDD/vehicles_sample.csv"
import pandas as pd
df = pd.read_csv(path)

In [ ]:
df.tail(10)

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '', regex=False) #eliminar mayusculas, espacios vacios y busca los espacios ' ' y los reemplaza por ''

In [ ]:
df.info()

# **Comentarios del Cientifico**
Hay varias columnas que no me sirven para la predicción. Entendiendo el contexto, busco realizar un modelo que pueda predecir el valor de venta de un vehiculo. Si bien, podria tener relevancia el lugar donde se vende, omitiré esas variables y haré un modelo "general".

In [ ]:
# Lista original de columnas a eliminar
cols_to_drop = [
    'id', 'url', 'region', 'region_url',
    'vin', 'size', 'image_url', 'description',
    'county', 'long', 'posting_date', 'lat'
]

# Filtramos solo las columnas que realmente están en el DataFrame
cols_presentes = [col for col in cols_to_drop if col in df.columns]

# Eliminamos solo esas
df = df.drop(columns=cols_presentes)

# Verificamos
print(df.head())

In [ ]:
df.info()

In [ ]:
df_limpio = df.dropna()

In [ ]:
df_limpio.info()

In [ ]:
def resumen_valores_unicos(df_limpio):   #cree una funcion para ver los valores unicos.
    for col in df_limpio.columns:
        print(f"Columna: {col}")
        print(f" - Tipo de dato: {df_limpio[col].dtype}")
        print(f" - Nº de valores únicos: {df_limpio[col].nunique()}")

        # Obtener y ordenar los valores únicos
        valores_ordenados = sorted(df_limpio[col].dropna().unique())

        # Mostrar hasta los primeros 10 valores únicos
        print(f" - Valores únicos (primeros 10): {valores_ordenados[:10]}")
        print("-" * 50)


In [ ]:
resumen_valores_unicos(df_limpio)

In [ ]:
df.to_csv("/content/drive/MyDrive/BBDD/vehicles_sample_limpio.csv", index=False)  #con esta ruta guardo el CSV. Por si ocurre algo.

In [ ]:
df_limpio.info()

In [ ]:
df_limpio = df_limpio.drop_duplicates() #eliminamos duplicados.

In [ ]:
print(f"Número de filas después de eliminar duplicados: {df_limpio.shape[0]}")

In [ ]:
print(df_limpio['fuel'].unique())
print(df_limpio['condition'].unique())

In [ ]:
from sklearn.preprocessing import MinMaxScaler #escalar caracteristicas numericas

escalador = MinMaxScaler()
columnas_numericas = ['price', 'year', 'odometer']

df_limpio[columnas_numericas] = escalador.fit_transform(df_limpio[columnas_numericas])

# **Visualizaciones UNIVARIADAS**

A continuación realizaré analisis visual con sus respectivos comentarios antes de eliminar cualquier tipo de outliers. Guardaré el df, y procesaré otro para guardar sin outliers y realizaré comparaciones de como afecta el modelaje el que no se eliminen los outliers de la data.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Distribución del precio
sns.histplot(df_limpio['price'], kde=True, bins=30)
plt.title('Distribución del Precio')
plt.xlabel('Precio')
plt.ylabel('Frecuencia')
plt.show()


**Comentarios del Cientifico:**
* La mayoria de los precios de los autos se concentran hasta $30.000 aprox. Esto indica que la mayoria de los autos del dataset son de gama baja o media. Validando con información que encontré internet, craiglist se caracteriza por realizar ventas de este tipo. Para ese sector.

In [ ]:
# Configurar el tamaño del gráfico
plt.figure(figsize=(10, 4))

# Crear el boxplot
sns.boxplot(x=df_limpio['price'], color='orange')

# Añadir título y etiquetas
plt.title('Boxplot del Precio')
plt.xlabel('Precio')

# Mostrar el gráfico
plt.show()

**Comentarios del Cientifico:**
* Se confirma que los precios de los autos están por debajo de los USD $30.000 dolares, sin embargo hay unos outliers, los que a mi criterio eliminaré del modelo.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.hist(df['year'], bins=30, edgecolor='black')
plt.title('Distribución de autos por año de fabricación')
plt.xlabel('Año')
plt.ylabel('Cantidad de autos')
plt.grid(True)
plt.show()

**Comentarios del Cientifico:**
* Se observan que existen autos muy antiguos, pero se centra más en autos entre el 2005 y 2020.

# **Visualizaciones MultiVariadas.**

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='odometer', y='price', data=df_limpio)
plt.title('Precio vs Kilometraje (Odometer)')
plt.xlabel('Kilometraje')
plt.ylabel('Precio')
plt.show()


**Comentario del Cientifico:**
* Como se observo en graficos univariados, hay precios de autos tipo outliers que se eliminarán. Y si, hay autos con muuuuuucho kilometraje, quizás hasta los regalan, por que craiglist tambien permutan cosas.

In [ ]:
plt.figure(figsize=(8, 5))
sns.heatmap(df_limpio[['price', 'year', 'odometer']].corr(), annot=True, cmap='coolwarm')
plt.title('Matriz de Correlación')
plt.show()


**Comentario del Cientifico:**

* Respecto a precio y año, hay una correlacion de 0.056, es una correlación positiva lo que afirma que autos nuevos tienden a tener un precio levemente más altos.
* Correlacion de cantidad de kilometros y precio es algo negativa, lo que indica que a mayor kilometraje, el precio tiende a bajar.
* La correlacion entre el precio y el año, tambien es negativa, lo que sugiere que los autos nuevos tienen menos kilemtraje. Ni modo comprar un auto nuevo con mucho kilometros.

# **MODELADO CON OUTLIERS**

In [ ]:
df_limpio = pd.get_dummies(df_limpio, drop_first=True) #transformacion de caracteristicas categoricas a numericas.

In [ ]:
df_limpio.info()

In [ ]:
X = df_limpio.drop('price', axis=1)
y = df_limpio['price']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Importar modelos y métricas
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Modelo 1: Regresión Lineal
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

# Modelo 2: Random Forest
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Función para evaluar modelos
def evaluar_modelo(y_true, y_pred, nombre):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    print(f"\n📊 Evaluación de {nombre}:")
    print(f"MSE:  {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R²:   {r2:.4f}")

# Evaluación de ambos modelos
evaluar_modelo(y_test, y_pred_lr, "Regresión Lineal")
evaluar_modelo(y_test, y_pred_rf, "Random Forest Regressor")


# **Guardaremos Ambos Modelos**

In [ ]:
import joblib
#guardamos los nombres de las columnas
joblib.dump(X_train.columns, "columnas_modelo.pkl")

# Guardar Regresión Lineal
joblib.dump(lr, "modelo_lineal.pkl")

# Guardar Random Forest
joblib.dump(rf, "modelo_random_forest.pkl")


# **Haremos unas pruebas....**
* utilizaré un auto año 2015, marca toyota, de gasolina, transmision automatica, en buenas condiciones y del tipo sedan.

Veamos con RadomForest.

In [ ]:
import pandas as pd
import joblib

# Cargar modelo entrenado y columnas originales
modelo = joblib.load("modelo_random_forest.pkl")
columnas = joblib.load("columnas_modelo.pkl")

# Crear input del usuario (ejemplo)
nuevo_auto = pd.DataFrame([{
    'year': 2015,
    'odometer': 80000,
    'manufacturer_toyota': True,
    'fuel_gasoline': True,
    'transmission_automatic': True,
    'condition_good': True,
    'type_sedan': True,
    # ... (todas las demás columnas dummy necesarias en False)
}], columns=columnas).fillna(False)

# Predecir
prediccion = modelo.predict(nuevo_auto)

# Mostrar resultado
print(f"💰 Precio estimado para el auto: ${prediccion[0]:,.2f}")


Ahora con el Modelo Lineal.

In [ ]:
import pandas as pd
import joblib

# Cargar modelo de regresión lineal y columnas originales
modelo_lineal = joblib.load("modelo_lineal.pkl")
columnas = joblib.load("columnas_modelo.pkl")

# Crear el input del usuario (idéntico al usado con Random Forest)
nuevo_auto = pd.DataFrame([{
    'year': 2015,
    'odometer': 80000,
    'manufacturer_toyota': True,
    'fuel_gasoline': True,
    'transmission_automatic': True,
    'condition_good': True,
    'type_sedan': True,
    # ... otras columnas dummy quedarán en False automáticamente
}], columns=columnas).fillna(False)

# Predecir con el modelo de regresión lineal
prediccion_lineal = modelo_lineal.predict(nuevo_auto)

# Mostrar resultado
print(f"📉 Precio estimado con Regresión Lineal: ${prediccion_lineal[0]:,.2f}")


In [ ]:
import pandas as pd
import joblib

def predecir_precios_vehiculos():
    # Cargar modelos y columnas
    columnas = joblib.load("columnas_modelo.pkl")
    modelo_lineal = joblib.load("modelo_lineal.pkl")
    modelo_rf = joblib.load("modelo_random_forest.pkl")

    # Lista para almacenar resultados
    historial = []

    while True:
        print("\n🔧 INGRESO DE DATOS DEL VEHÍCULO")

        # Entradas numéricas
        year = int(input("Año del vehículo: "))
        odometer = float(input("Kilometraje (odometer): "))

        # Diccionario base
        entrada = {'year': year, 'odometer': odometer}

        # Entradas categóricas (dummy)
        opciones_true = [
            input("Fabricante (ej: manufacturer_toyota): "),
            input("Tipo de combustible (ej: fuel_gasoline): "),
            input("Transmisión (ej: transmission_automatic): "),
            input("Condición (ej: condition_good): "),
            input("Tipo de auto (ej: type_sedan): ")
        ]

        for col in opciones_true:
            if col in columnas:
                entrada[col] = True  # Solo marcamos True si la columna existe

        # Crear DataFrame para predecir
        nuevo_auto = pd.DataFrame([entrada], columns=columnas).fillna(False)

        # Predecir con ambos modelos
        pred_lineal = modelo_lineal.predict(nuevo_auto)[0]
        pred_rf = modelo_rf.predict(nuevo_auto)[0]

        # Mostrar resultados
        print("\n📈 RESULTADOS DE PREDICCIÓN:")
        print(f"🔹 Regresión Lineal:     ${pred_lineal:,.2f}")
        print(f"🔹 Random Forest:        ${pred_rf:,.2f}")

        # Guardar en historial
        entrada_resumen = entrada.copy()
        entrada_resumen['Pred_LR'] = pred_lineal
        entrada_resumen['Pred_RF'] = pred_rf
        historial.append(entrada_resumen)

        # ¿Desea hacer otra?
        continuar = input("\n¿Deseas hacer otra predicción? (s/n): ").lower()
        if continuar != 's':
            break

    # Mostrar resumen
    print("\n📋 Historial de predicciones:")
    df_historial = pd.DataFrame(historial)
    print(df_historial[['year', 'odometer', 'Pred_LR', 'Pred_RF'] + [col for col in df_historial.columns if col not in ['year', 'odometer', 'Pred_LR', 'Pred_RF']]].to_string(index=False))

    # Opcional: guardar en archivo CSV
    guardar = input("\n¿Deseas guardar el historial en un archivo CSV? (s/n): ").lower()
    if guardar == 's':
        df_historial.to_csv("historial_predicciones_autos.csv", index=False)
        print("✅ Historial guardado como 'historial_predicciones_autos.csv'.")

# Ejecutar función
predecir_precios_vehiculos()


# **Copia del DF y procesamiento de Outliers.**

In [ ]:
df_limpio2 = df_limpio.copy() #copia del DF

In [ ]:
# Configurar el tamaño del gráfico
plt.figure(figsize=(10, 4))

# Crear el boxplot
sns.boxplot(x=df_limpio2['price'], color='orange')

# Añadir título y etiquetas
plt.title('Boxplot del Precio')
plt.xlabel('Precio')

# Mostrar el gráfico
plt.show()

In [ ]:
df_limpio2.info()

In [ ]:
# Calcular cuartiles
Q1 = df_limpio2['price'].quantile(0.25)
Q3 = df_limpio2['price'].quantile(0.75)
IQR = Q3 - Q1

# Límites para detectar outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Crear nuevo DataFrame sin outliers
df_sin_outliers = df_limpio2[(df_limpio2['price'] >= limite_inferior) & (df_limpio2['price'] <= limite_superior)]

# Mostrar resultados
print(f"Número de filas antes: {len(df_limpio2)}")
print(f"Número de filas después de eliminar outliers: {len(df_sin_outliers)}")


In [ ]:
# Configurar el tamaño del gráfico
plt.figure(figsize=(10, 4))

# Crear el boxplot
sns.boxplot(x=df_sin_outliers['price'], color='orange')

# Añadir título y etiquetas
plt.title('Boxplot del Precio')
plt.xlabel('Precio')

# Mostrar el gráfico
plt.show()

# **Seleccion de Features y Target, modelo sin Outliers.**

In [ ]:
X2 = df_sin_outliers.drop(columns='price')
y2 = df_sin_outliers['price']

In [ ]:
from sklearn.model_selection import train_test_split

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression

modelo_lr2 = LinearRegression()
modelo_lr2.fit(X2_train, y2_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

modelo_rf2 = RandomForestRegressor(n_estimators=100, random_state=42)
modelo_rf2.fit(X2_train, y2_train)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluar_modelo2(nombre, y_verdadero, y_predicho):
    print(f"\n--- {nombre} ---")
    print("MAE:", mean_absolute_error(y_verdadero, y_predicho))
    print("RMSE:", np.sqrt(mean_squared_error(y_verdadero, y_predicho)))
    print("R2:", r2_score(y_verdadero, y_predicho))

# Predicciones
y2_pred_lr = modelo_lr2.predict(X2_test)
y2_pred_rf = modelo_rf2.predict(X2_test)

# Evaluaciones
evaluar_modelo2("Linear Regression 2", y2_test, y2_pred_lr)
evaluar_modelo2("Random Forest 2", y2_test, y2_pred_rf)


# **Grafica Predicciones Vs Valores Reales**

In [ ]:
import matplotlib.pyplot as plt

def graficar_predicciones(y_test, y_pred, titulo):
    plt.figure(figsize=(8, 6))
    plt.scatter(y_test, y_pred, alpha=0.3)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r')
    plt.xlabel('Precio Real')
    plt.ylabel('Precio Predicho')
    plt.title(titulo)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Gráficos
graficar_predicciones(y2_test, y2_pred_lr, 'Regresión Lineal 2: Precio Real vs Predicho')
graficar_predicciones(y2_test, y2_pred_rf, 'Random Forest 2: Precio Real vs Predicho')


**Guardamos el modelo sin outliers.**

In [ ]:
import joblib

# Guardar los modelos entrenados
joblib.dump(modelo_lr2, 'modelo_lr2.pkl')
joblib.dump(modelo_rf2, 'modelo_rf2.pkl')
# Obtener columnas del modelo sin outliers
columnas_modelo2 = X2.columns.tolist()
# Guardar en un archivo .pkl
joblib.dump(columnas_modelo2, "columnas_modelo2.pkl")

In [55]:
import pandas as pd
import joblib

def predecir_precios_vehiculos2():
    # Cargar modelos y columnas
    columnas2 = joblib.load("columnas_modelo2.pkl")
    modelo_lineal2 = joblib.load("modelo_lr2.pkl")
    modelo_rf2 = joblib.load("modelo_rf2.pkl")

    # Lista para almacenar resultados
    historial2 = []

    while True:
        print("\n🔧 INGRESO DE DATOS DEL VEHÍCULO (Modelo sin outliers)")

        # Entradas numéricas
        year = int(input("Año del vehículo: "))
        odometer = float(input("Kilometraje (odometer): "))

        # Diccionario base
        entrada2 = {'year': year, 'odometer': odometer}

        # Entradas categóricas (dummy)
        opciones_true2 = [
            input("Fabricante (ej: manufacturer_toyota): "),
            input("Tipo de combustible (ej: fuel_gasoline): "),
            input("Transmisión (ej: transmission_automatic): "),
            input("Condición (ej: condition_good): "),
            input("Tipo de auto (ej: type_sedan): ")
        ]

        for col in opciones_true2:
            if col in columnas2:
                entrada2[col] = True  # Solo marcamos True si la columna existe

        # Crear DataFrame para predecir
        nuevo_auto2 = pd.DataFrame([entrada2], columns=columnas2).fillna(False)

        # Predecir con ambos modelos
        pred_lineal2 = modelo_lineal2.predict(nuevo_auto2)[0]
        pred_rf2 = modelo_rf2.predict(nuevo_auto2)[0]

        # Mostrar resultados
        print("\n📈 RESULTADOS DE PREDICCIÓN (sin outliers):")
        print(f"🔹 Regresión Lineal:     ${pred_lineal2:,.2f}")
        print(f"🔹 Random Forest:        ${pred_rf2:,.2f}")

        # Guardar en historial
        entrada_resumen2 = entrada2.copy()
        entrada_resumen2['Pred_LR2'] = pred_lineal2
        entrada_resumen2['Pred_RF2'] = pred_rf2
        historial2.append(entrada_resumen2)

        # ¿Desea hacer otra?
        continuar2 = input("\n¿Deseas hacer otra predicción? (s/n): ").lower()
        if continuar2 != 's':
            break

    # Mostrar resumen
    print("\n📋 Historial de predicciones:")
    df_historial2 = pd.DataFrame(historial2)
    print(df_historial2[['year', 'odometer', 'Pred_LR2', 'Pred_RF2'] +
          [col for col in df_historial2.columns if col not in ['year', 'odometer', 'Pred_LR2', 'Pred_RF2']]].to_string(index=False))

    # Opcional: guardar en archivo CSV
    guardar2 = input("\n¿Deseas guardar el historial en un archivo CSV? (s/n): ").lower()
    if guardar2 == 's':
        df_historial2.to_csv("historial_predicciones_autos2.csv", index=False)
        print("✅ Historial guardado como 'historial_predicciones_autos2.csv'.")

# Ejecutar función
predecir_precios_vehiculos2()



🔧 INGRESO DE DATOS DEL VEHÍCULO (Modelo sin outliers)
Año del vehículo: 2026
Kilometraje (odometer): 0
Fabricante (ej: manufacturer_toyota): manufacturer_nissan
Tipo de combustible (ej: fuel_gasoline): fuel_hybrid
Transmisión (ej: transmission_automatic): transmission_automatic
Condición (ej: condition_good): condition_excelent
Tipo de auto (ej: type_sedan): type_sedan


/tmp/ipython-input-55-2052203742.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  nuevo_auto2 = pd.DataFrame([entrada2], columns=columnas2).fillna(False)



📈 RESULTADOS DE PREDICCIÓN (sin outliers):
🔹 Regresión Lineal:     $10,137.55
🔹 Random Forest:        $18,371.11

¿Deseas hacer otra predicción? (s/n): n

📋 Historial de predicciones:
 year  odometer     Pred_LR2  Pred_RF2  manufacturer_nissan  fuel_hybrid  type_sedan
 2026       0.0 10137.549532  18371.11                 True         True        True


KeyboardInterrupt: Interrupted by user

# **CONCLUSION FINAL**

**1. Con outliers:**
* Ambos modelos tuvieron bajo rendimiento.
* El R² fue negativo en regresión lineal, lo que indica que el modelo era peor que simplemente predecir la media.
* Random Forest apenas logró un R² de 0.03, sin poder explicar la varianza real.

**2. Sin outliers:**
* El rendimiento mejoró sustancialmente.
* La regresión lineal subió a un R² de 0.22, lo que indica que ahora sí logra capturar cierta tendencia.
* El modelo Random Forest pasó a un excelente R² de 0.6156, indicando un buen nivel de predicción.
* También bajaron drásticamente los errores (MAE y RMSE), lo que mejora la confiabilidad de las predicciones.